# 캘리포니아 주택 가격 예측

### 기본 정보

- 인구, 중간소득 등의 특성을 사용하여 주택 가격 예측
- 다중 회귀, 단방향 회귀(반대 : 다변량 회귀)
    - 다중 회귀 : 여러 개의 특성을 이용한 회귀
    - 단방향 회귀 : 하나의 데이터에 대한 예측
    - 다변량 회귀 : 여러 개의 수치를 예측하기 위한 회귀
- 회귀의 성능 측정 방법
    - 평균 제곱근의 오차를 활용
    - 평균 절대오차를 활용

- 일차적으로 파이썬 커널을 설정하면서 정상적으로 import가 되는지 체크해주었다. 
- 커널에 원하는 import 추가하는 법
    1. `venv`로 커널을 생성해준다. 
    2. `source .venv/bin/activate`로 venv를 활성화해준다. 
    3. 내부에서 `pip install`로 원하는 라이브러리를 다운로드 받는다. 

In [72]:
import sys

assert sys.version_info >= (3, 5)

import sklearn

assert sklearn.__version__ >= "0.20"

import numpy as np
import os

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

ROOT_DIR = '.'
CHAPTER_ID = 'end_to_end_project'
IMAGE_PATH = os.path.join(ROOT_DIR, 'images', CHAPTER_ID)
os.makedirs(IMAGE_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension = 'png', resolution = 100): 
    path = os.path.join(IMAGE_PATH, fig_id + '.' + fig_extension)
    print('그림 저장 : ', fig_id)

    if tight_layout:
        plt.tight_layout()
    
    plt.savefig(path, format=fig_extension, dpi=resolution)

import warnings

warnings.filterwarnings(action='ignore', message='^internal gelsd')


# 데이터 가져오기

In [73]:
import os
import tarfile
import urllib.request

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/rickiepark/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()


In [ ]:

fetch_housing_data()

In [ ]:
import pandas as pd

csv_path = os.path.join(HOUSING_PATH, "housing.csv")

pd.read_csv(csv_path)

In [ ]:
pd.read_csv(csv_path).head(10)

- 해당 데이터는 대부분 수치로 되어있다. 
- longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, median_house_value, ocean_proximity가 존재하고, dictionary구조로 데이터를 뽑아내들 수 있다. 

In [77]:
housing = pd.read_csv(csv_path)

- pandas를 통해 읽어온 데이터셋은 `describe()`를 이용해 데이터 요약본을 알 수 있다. 하지만, 이는 숫자형 데이터만 조회가능

In [ ]:
housing.describe()

In [ ]:
import matplotlib.pyplot as plt

housing.hist(bins=50, figsize=(20, 15))
save_fig('attribute_histogram_plots')

plt.show()

- 가우시안 분포가 되지 못하는 경우가 다양하다. 이런 데이터들을 전처리를 통해 정규분포화 시켜야 한다. 
- 각 분포별 x축이 다르다. 이를 일정하게 맞추는 작업이 추후 필요할 것 같다. 

# TestSet, TrainingSet 분리

- 테스트와 훈련 데이터가 중복될 경우, 과적합이 발생할 수 있다.
- 이를 방지하기 위해 훈련 데이터셋과 테스트 데이터셋을 사전에 분리한다. 

In [80]:
import numpy as np

# 실제로 scikit-learn에서 테스트 데이터를 분리해주는 함수가 존재한다 sklearn.train_test_split()
def split_train_set(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [81]:
train_set, test_set = split_train_set(housing, 0.2)

In [82]:
from zlib import crc32

def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32

def split_train_set_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_:test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

- 실제 개발 중 데이터셋에 변경이 있을 수 있다. 이 때 데이터를 다시 받을 수 있는 로직을 호출하면 된다. 

In [83]:
housing_with_id = housing.reset_index()
train_set, test_set = split_train_set_by_id(housing_with_id, 0.2, "index")

In [84]:
import sklearn.model_selection


train_set, test_set = sklearn.model_selection.train_test_split(housing, test_size=0.2, random_state=42)

- 결국 sklearn쓰는게 제일 편하다

In [ ]:
test_set.head() # 인덱스가 다 섞임

# 예제 - median_income

- median_income은 연속적인 값이다. 
- 이를 categorized factor로 만들어주어야 한다. 
- 특정 범주를 정해 라벨링을 해주는 방식으로 카테고리화 시킨다. 

In [86]:
housing['income_cat'] = pd.cut(
    housing['median_income'],
    bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
    labels=[1, 2, 3, 4, 5]
)

In [ ]:
housing['income_cat'].value_counts()

In [ ]:
housing['income_cat'].hist()

In [89]:
import sklearn.model_selection as model_selection

split = model_selection.StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in split.split(housing, housing['income_cat']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [ ]:
strat_test_set['income_cat'].value_counts() / len(strat_test_set)

In [ ]:
housing['income_cat'].value_counts() / len(housing)

- 사실 stratify로 train_test_split해주는 로직은 sklearn.model_selection에 이미 존재한다.

In [92]:
st_train_set, st_test_set = model_selection.train_test_split(housing, test_size=0.2, random_state=42, stratify=housing['income_cat'])

In [ ]:
st_train_set['income_cat'].value_counts() / len(st_train_set)

In [94]:
for set_ in (strat_test_set, strat_train_set):
    set_.drop('income_cat', axis=1, inplace=True)

# 데이터이해를 위한 탐색과 시각화

In [95]:
housing_traing = strat_train_set.copy()

In [ ]:
housing_traing.plot(kind='scatter', x='longitude', y='latitude')

In [ ]:
housing_traing.plot(kind='scatter', x='longitude', y='latitude', alpha=0.1)


- scatter plot으로 전체의 분포를 점으로 찍을 수 있다. 
- 그런데 alpha로 조금 더 진한 부분이 선택적으로 보이게 된다. 
- `alpha`를 이용해 투명도를 설정할 수 있다. 이를 통해 빈도가 높은 곳은 진하게 확인할 수 있다. 

In [ ]:
housing_traing.plot(
    kind='scatter', 
    x='longitude', 
    y='latitude', 
    alpha=0.4, 
    s=housing_traing['population']/100, 
    label='population', 
    figsize=(10, 7), 
    c='median_house_value',
    cmap=plt.get_cmap('jet'), 
    colorbar=True, 
    sharex=False)
plt.legend()


- 현재의 구조는 위도, 경도를 통해 생성되어있는 구조이다. 
- 이를 지도 위에 표기해보자. 

In [ ]:
image_path = os.path.join(ROOT_DIR, 'images', 'end_to_end_project')
os.makedirs(image_path, exist_ok=True)
filename = 'california.png'
url = DOWNLOAD_ROOT + 'images/end_to_end_project/' + filename
urllib.request.urlretrieve(url, os.path.join(image_path, filename))

In [ ]:
import matplotlib.image as mpimg

california_img = mpimg.imread(os.path.join(image_path, filename))
ax = housing_traing.plot(
    kind='scatter', 
    x='longitude', 
    y='latitude', 
    alpha=0.4, 
    s=housing_traing['population']/100, 
    label='population', 
    figsize=(10, 7), 
    c='median_house_value',
    cmap=plt.get_cmap('jet'), 
    colorbar=False)

plt.imshow(california_img, extent=[-124.55, -113.80, 32.45, 42.05], alpha=0.5, cmap=plt.get_cmap('jet'))

plt.ylabel('Latitude', fontsize = 14)
plt.xlabel('Longitude', fontsize = 14)

prices = housing_traing['median_house_value']
tick_values = np.linspace(prices.min(), prices.max(), 11)
cbar = plt.colorbar(ticks=tick_values/prices.max())
cbar.ax.set_yticklabels(['$%dk'%(round(v/1000)) for v in tick_values], fontsize=14)
cbar.set_label('Median House Value', fontsize=14)

plt.legend(fontsize=16)
plt.show()


- 지금까지는 데이터를 시각화해 인사이트를 얻기 위한 과정이었다. 
- 이번엔 각 데이터간의 상관관계를 찾아야 한다.
- 이를 위해 상관관계 행렬을 얻기 위한 함수인 `corr()`이 존재한다. 
- 현재 버전에서, 상관행렬 생성시 String 타입은 행렬화시킬 수 없다.
- `pandas.get_dummies`를 이용해 처리하자. 
    - `get_dummies` : 문자열형의 categorized data를 numerical data로 처리한다. 
    - `HANDS ON MACHINE LEARNING 2판`에서는 해당 로직이 바로 된다는 강의였지만, 불가능!
    - 그런데, 지금과 같은 경우 문자열이 바로 처리되지만, 그렇지 못하는 nullable한 케이스는 고려할 필요 없을까?

In [101]:
corr_matrix = pd.get_dummies(housing_traing, drop_first=True).corr()

In [ ]:
corr_matrix['median_house_value'].sort_values(ascending=False)

- 상관계수는 0에 가까울 수록 의미가 없고, 1에 가까울 수록 관계가 큰 것을 의미한다. 

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ['median_house_value', 'median_income', 'total_rooms', 'housing_median_age']
exclude_attributes = ['ocean_proximity']

scatter_matrix(housing_traing[attributes], figsize=(12, 8))


- 총 4개의 데이터를 이용해 만들 수 있는 모든 경우의 수에 대한 scatter plot을 표현한 상태이다. 
- x와 y가 같은 scatter plot은 histogram으로 대체된다. 

In [ ]:
housing_traing.plot(kind='scatter', x='median_income', y='median_house_value', alpha = 0.1)
plt.axis([0, 16, 0, 550000])

- 위의 데이터중, 특정 데이터를 조회해 디테일을 볼 수 있다. 
- median_income, median_house_value는 직선형 구조임을 알 수 있다. 

## 수동으로 새로운 특성을 추가하기

In [ ]:
housing_traing = housing_traing.drop(columns=['rooms_per_household', 'bedroom_per_room', 'population_per_household'])

housing_traing['rooms_per_household'] = housing_traing['total_rooms'] / housing_traing['households']
housing_traing['bedroom_per_room'] = housing_traing['population'] / housing_traing['households']
housing_traing['population_per_household'] = housing_traing['population'] / housing_traing['households']

attributes.append('rooms_per_household')
attributes.append('bedroom_per_room')
attributes.append('population_per_household')

In [ ]:
attributes = ['median_house_value', 'median_income', 'total_rooms', 'housing_median_age', 'rooms_per_household', 'bedroom_per_room', 'population_per_household', 'latitude', 'longitude']

corr_matrix = housing_traing[attributes].corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

- 조회결과, `rooms_per_household`가 생각보다 높은 상관계수를 가지고 있다. 
- 이에 대한 산점도를 조회해본다. 

In [ ]:
housing_traing.plot(kind='scatter', x='rooms_per_household', y='median_house_value', alpha=0.1)
plt.show()

- 아주 미세하게 기울어진 선형임을 알 수 있다. 
- 이걸 어떻게 활용해야 할까?